### Learning Algorithm ###
The DQN(Deep QNetwork) algorithm approximates an optimal Q value function $q_*$ mapping state action pairs to their values. This function is represented as a Neural Network whose weights $w$ are learnt as the agent interacts with its environment. Each interaction is represented as a tuple
$$<s, a, r, s'>$$
-  $s$ is the current state observed from the environment
-  $a$ is the action chosen by the agent based on an $\epsilon$-greedy policy
-  $r$ is the reward collected by following action $a$ in state $s$
-  $s'$ is the next state reached by by following action $a$ in state $s$

The successive interactions are stored in a replay buffer. The agent samples periodically a minibatch of interactions from the buffer. The use of this replay buffer prevents correlations between successive interactions. The agent uses the sampled interactions to compute a loss function and to update the weights of its networks using gradient descent. To this end, the agent is equipped with a double Q-Value network (DDQN) composed of a local network (the main one whose weights are updated using gradient descent) and a target network (a mirror of the local network that is updated more slowly) in charge of providing stable targets. The weights of the local network are updated as follows : 
$$\Delta w = \alpha * [R + \gamma * max_{a} \hat{q}(s', a, w^{-}) - \hat{q}(s, a, w)] * \nabla \hat{q}(s, a, w)$$
where :
- $\hat{q}(s, a, w)$ is the estimate of the qvalue for the current state $s$ and chosen action $a$ provided by the local network (weights $w$)
- $\hat{q}(s', a, w^{-})$ is the estimate of the qvalue for the next state $s'$ and an action $a$ provided by the target network (weights $w^{-}$)
- $R + \gamma * max_{a} \hat{q}(s', a, w^{-}) - \hat{q}(s, a, w)$ is the loss or temporal difference (TD) between the current estimate and the target which is the sum of the current reward and the discounted qvalue of the next state. 

The weights $w^{-}$ of the target network follow the updates of the $w$ more slowly (soft updates):
    $$w^{-} = \tau * w + (1 - \tau) * w^{-}$$
Without these more stable targets, we would encounter a harmful form of correlation whereby we shift the weights of the network based on constantly moving targets.


### Hyperparameters
- BUFFER_SIZE : the capacity of the replay buffer - a store of samples $<s, a, r, s'>$
- BATCH_SIZE : the minibatch size that is sampled from the replay buffer and used to train the local network
- GAMMA : factor used to discount the future rewards
- TAU : factor used for the soft updates of the weights in the target network
- LR : learning rate
- UPDATE_EVERY : periodicity in steps : how often the weights are updated
```python
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate
UPDATE_EVERY = 4        # how often to update the network
````
- n_episodes (2000) : number of episodes required to train an agent where each episode is a sequence of interactions between the agent and its environment. An episode has a maximal length or can be ended by the environment (flag done)
- max_t (1000) : maximal length of an episode
- epsilon : parameter for $\epsilon$-greedy policy. With probability $\epsilon$, the agent chooses an action at random (exploration) or with probability 1 - $\epsilon$ chooses the best estimate for action (exploitation).The actual value of $\epsilon$ decreases exponentially (see following parameters) to allow full exploration when learning starts and minimal exploration when learning ends.
- eps_start (1.0) : full exploration, value of $\epsilon$ when learning starts
- eps_end (0.01) : minimal exploration, value of $\epsilon$ after a sufficient number of leraning interactions
- eps_decay (0.995) : rate of exponential decay for $\epsilon$

### Plot of Rewards
The following plot shows the evolution of the scores during the training phase. A score average of 15 over the last 100 episodes has been considered enough to end the learning phase. More specifically, the environment has been solved in 649 episodes with an average score of 15.04 over the last 100 episodes.  
![](scores.jpg) 

### Ideas for Future Work
- Tuning of hyperparamaters : in particular the learning rate, the minibatch size and epsilon
- Prioritized sampling : priorities are attached to the interactions that reflect their temporal differences and the interactions are sampled according to these priorities (i.e. non uniformly) but this requires also an adaptation to the loss function to prevent bias that could originate from a non uniform distribution.
- Dueling networks: implement a network producing two streams, the value of the state and the advantage of an action where the resulting q-value is a combination of the state value and of the advantage of the action.